In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
import pandas as pd
import torch

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image


class TagwalkDataset(Dataset):
    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        self.reference_dataset = self.read_reference_dataset(csv_path)
        print self.reference_dataset.head()
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = self.reference_dataset['image']
        self.y_train = self.mlb.fit_transform(
            self.reference_dataset['tags']
        ).astype(np.float32)

    def read_reference_dataset(self, csv_path):
        tmp_df = pd.read_csv(csv_path)
        
        tmp_df = (
            tmp_df
            .groupby('image')['tag']
            .apply(list)
        ).reset_index()
        tmp_df.columns = ['image', 'tags']
        return tmp_df
        
    def __getitem__(self, index):
        item_img_path = '/'.join([
            self.img_path,
            self.X_train[index]
        ])
        img = Image.open(item_img_path)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [ ]:
BASE_DATA = '/Volumes/bobby/tag_walk/tag_walk/data/tag_walk/'



tw_dataset = TagwalkDataset(
    csv_path=BASE_DATA + 'assocs.csv',
    img_path=BASE_DATA + 'images/all',
    img_ext='jpg',
    transform=(
)
)

In [ ]:
tw_dataset[0][0]

In [2]:
from fachung.datasets.tagwalk import TagwalkDataset

In [5]:
tw_dataset = TagwalkDataset()

/Volumes/bobby//tag_walk/tag_walk//data/tag_walk/
                          image  \
0  001 Valentino PAP FW1617.jpg   
1     001-LEONARD_PREFALL17.jpg   
2                       001.jpg   
3               001ALF_0016.JPG   
4              001_CSC_0016.jpg   

                                                tags  
0  [army, black, coat, double-breast, long-coat, ...  
1  [art, blue, blue-pants, blue-trousers, floral,...  
2  [beige, black, bustier, dominatrix, dress, flo...  
3  [1960, blue, bomber, mini-skirt, navy-blue, na...  
4          [brown, dance, dress, lace, pearls, silk]  


In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(tw_dataset,
                          batch_size=256,
                          shuffle=True,
                          num_workers=4)

In [8]:
train_loader